#  TrafficGeneration
<i>Adapted for use with FABRIC from [Traffic Generation](https://www.cs.unc.edu/Research/geni/geniEdu/08-TrafficGen.html)</i>
        
<b> Prerequisites  </b>
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-3 at https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/)

In This Experiment you will be introduced to the principles of traffic generation using Nping. This is the second step in this assignment, to go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)

## 1. Design the Experiment
### 1.1 Reserve Resources

#### Import the Fabric API

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,7a5adb91-c4c0-4a1c-8021-7b6c56af196f
Bastion Username,lrco247_0000050326
Bastion Private Key File,/home/fabric/work/fabric_config/testingNEWkey
Bastion Host,bastion-1.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


####  Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.


In [2]:
slice_name = "TrafficGeneration"
try:
    slice = fablib.get_slice(slice_name)
    print(f"Slice: {slice.get_name()}, {slice.get_state()}")
except Exception as e:
    print(f"Get Slices Fail: {e}")

Slice: TrafficGeneration, StableOK


## 1.2 Setup the node network
### 1.2.1 Give each node an address

In [3]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# configure nodes
try:
    A = slice.get_node('ND1')
    B = slice.get_node('ND2')
    
    subnet = IPv4Network("10.0.0.0/24")
    
    #A's lan
    A_iface = A.get_interface(network_name="Lan")  
    A_iface.ip_addr_add(addr='10.0.0.1', subnet=subnet)
    #B's Lan
    B_iface = B.get_interface(network_name="Lan")  
    B_iface.ip_addr_add(addr='10.0.0.2', subnet=subnet)
    
    for node in slice.get_nodes():        
        stdout, stderr = node.execute(f'sudo ip link set dev ens7 up')
        
except Exception as e:
    print(f"Fail: {e}")

### 1.2.2 setting up the enviorment
This includes installing nping in both nodes, iperf, and R-studio for the assignment

In [4]:
try:
    for node in slice.get_nodes():        
        node.execute("sudo apt update")   
        node.execute("sudo apt-get install nmap -y")
        node.execute("sudo apt-get install iperf -y")
        #Graphics
        node.upload_file("./scripts/TG-data-analysis.R","TG-data-analysis.R")
        node.upload_file("./scripts/TG-output.sh","TG-output.sh")
        node.execute("sudo apt-get -y install moreutils r-base-core r-cran-ggplot2 r-cran-littler;sudo sysctl -w net.ipv4.tcp_no_metrics_save=1;chmod +x TG-output.sh ")
except Exception as e:
    print(f"Fail: {e}")

    

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
 

 Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2297 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [366 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [13.0 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1939 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [271 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/rest

### 2 Experiment
#### 2.1 Running Nping
1. Open an ssh terminal to each of the node

Nping comes from the Nmap package and is self contained, we will use this to generate traffic across the nodes. nping has different methods of running traffic, we will use tcp traffic using "--tcp" or udp with "--udp"

2. We can start by running a simple test on the network with the following command ```sudo nping --tcp -p 5000 10.0.0.2```

however we can also specify how many rounds we want to use with the "-c" option, can specify the length of the package "--data-length" and delay between sent packets "delay"

3. For this experiment we will run traffic generation using TCP traffic on the port 5000,for 4000 rounds, and we will start with data length of 1 and no delay, the command we will use is
   <br> ```sudo nping --tcp --dest-ip 10.0.0.2 -p 5000 --privileged -c 40000 --data-length 1024 --delay 0 ```
4. For more information about Nping: https://nmap.org/nping/
        


## Continue to The Assignment notebook

Once You have completed this notebook you should be able to continue to the Traffic Generation Assignment Notebook, you can ether open it on the explorer or click [Here](./Assignment.ipynb) to open the next notebook.